In [1]:
import os, sys
sys.path.append('../')
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import torch
import torchvision
import torchvision.transforms as transforms
from datasets.coco import CocoDataset
from utils.transforms import PhotometricDistort, Flip, Zoomout, \
                             ImageToTensor, CategoryToTensor, \
                             BoxToTensor, Resize, Normalize, \
                             CocoBoxToFracBoundaryBox, AssignObjToPbox
from functools import partial
import torch.nn as nn
import torch.nn.functional as F
from model.detector import SSD300
from loss.multibox import Loss
from torch.utils.tensorboard import SummaryWriter

DEVICE = 'cpu' if torch.cuda.device_count() == 0 else 'cuda:0'
print(DEVICE)

/Users/jcheng/opt/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


cpu


In [2]:
# initialize network 
N_CLASSES = 81
ssd = SSD300(n_classes=N_CLASSES, device=DEVICE)

In [3]:
# transforms
IMG_SZ = 300

distort_tfms  = [PhotometricDistort(0.5),
                 Flip(0.5)]
zoom_tfms     = [Zoomout(0.2, max_scale=2.5)]
toTensor_tfms = [ImageToTensor(), 
                 CategoryToTensor(), 
                 BoxToTensor()]
resize_tfms   = [Resize((IMG_SZ, IMG_SZ))]
target_tfms   = [CocoBoxToFracBoundaryBox(),
                 AssignObjToPbox(pboxes=ssd.pboxes, threshold=0.5)]
imgNorm_tfms  = [Normalize()]

tfms = transforms.Compose(distort_tfms + toTensor_tfms + zoom_tfms + imgNorm_tfms + resize_tfms + target_tfms)

# create dataset & dataloader
ds = CocoDataset('../datasets/', 'samples', 'instances', transforms=tfms)
assert ssd.n_classes == len(ds.id2cat.keys()), "# of class mismatch between dataset and ssd"
dl = torch.utils.data.DataLoader(ds, batch_size=4, shuffle=True, num_workers=2,
                                 collate_fn=partial(ds.collate_fn, img_resized=True))

# create tensorboard summary writer
writer = SummaryWriter('runs/vgg_base')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [4]:
# loss function
loss = Loss(neg_pos_ratio=3, 
            alpha=1.,
            device=DEVICE)

In [5]:
# data object
class Data():
    def __init__(self, train_dl, valid_dl):
        self.train_dl = train_dl
        self.valid_dl = valid_dl

In [6]:
# create a data obj that wraps around train/valid dls
data = Data(train_dl=dl, valid_dl=dl)

In [7]:
# learner
class Learner():
    """
    Base learner class for all other learners to inherit from
    """
    def __init__(self, model, loss_func, metrics, data):
        self.model, self.loss_func, self.metrics, self.data = model, loss_func, metrics, data

learner = Learner(model=ssd, loss_func=loss, metrics=None, data=data)

In [8]:
# experiment class
from exp.supervised import SupervisedExp
from exp.callbacks.setup_callback import SetupCallback
from exp.callbacks.cocodatabatch_callback import CocoDataBatchCallback
from exp.callbacks.tensorboard_callback import TensorBoardCallback
exp = SupervisedExp(learner, cb_funcs=[CocoDataBatchCallback, TensorBoardCallback])

/Users/jcheng/opt/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [9]:
# optimizer
EPOCHS = 10
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4

# init optimizer
opt = torch.optim.SGD(params=[{'params': exp.biases}, {'params': exp.weights}],
                      lr=1e-3, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

In [10]:
# train
exp.run(EPOCHS, opt)

location loss: 0.8708889484405518
class loss: 1.1888632774353027
location loss: 0.8758167028427124
class loss: 1.258622407913208
location loss: 0.8076287508010864
class loss: 0.6768227219581604
location loss: 0.824021577835083
class loss: 1.257200002670288
location loss: 0.6390993595123291
class loss: 1.2073030471801758
location loss: 0.647379457950592
class loss: 0.6691915988922119
location loss: 0.9161425232887268
class loss: 1.2270370721817017
location loss: 0.888397216796875
class loss: 1.2040914297103882
location loss: 0.4557616412639618
class loss: 0.6705176830291748
location loss: 0.807883620262146
class loss: 1.2396870851516724
location loss: 0.918779194355011
class loss: 1.1916431188583374
location loss: 0.7159225940704346
class loss: 0.6306052207946777
location loss: 0.6906858682632446
class loss: 1.2237944602966309
location loss: 0.7917832732200623
class loss: 1.1624113321304321
location loss: 0.793585479259491
class loss: 0.622937023639679
location loss: 1.1006864309310913


In [ ]:
!